In [5]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser

load_dotenv()

llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

In [9]:
from pydantic import BaseModel
from typing import Literal

class llm_schema(BaseModel):
    movie_summary_flag: Literal["positive","negative"]

llm_structured_output = llm.with_structured_output(llm_schema)

In [10]:
llm_structured_output.invoke("The movie was great")

llm_schema(movie_summary_flag='positive')

In [11]:
result = llm_structured_output.invoke("The movie was great")
result.model_dump()

{'movie_summary_flag': 'positive'}

# CHAIN with Conditional Chains

In [6]:
# task - 1 [prompt]

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie review evaluator."),
    ("human", "Please categorize the movie review as positive or negative : {input}")
])

In [7]:
# task - 2 [LLM]

llm_structured_output = llm.with_structured_output(llm_schema)

In [12]:
# task - 3 [Custom Runnable]

from langchain_core.runnables import RunnableLambda

def pydantic_json(input:llm_schema)-> str:
    
    return input.model_dump()['movie_summary_flag']

pydantic_json_lambda = RunnableLambda(pydantic_json)

### Conditional Chain 1

In [13]:
# task - 1 [prompt]

linkedin_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a LinkedIn post generator."),
    ("human", "Create a post for the following text for LinkedIn : {text}")
])

# task - 2 [LLM]

llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

# task - 3 [Str parser]

parser = StrOutputParser()


chain_linkedin = linkedin_prompt | llm | parser

### Conditional Chain 2

In [16]:
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnableBranch

In [21]:
def insta_chain(text:str):

    # task - 1 [prompt]
    insta_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a Instagram post generator."),
        ("human", "Create a post for the following text for Instagram : {text}")
        ])

    # task - 2 [LLM]
    llm = ChatGroq(temperature=0.7, model="llama-3.3-70b-versatile", api_key=os.getenv("GROQ_API_KEY"))

    # task - 3 [Str parser]
    parser = StrOutputParser()
    
    chain_insta = insta_prompt | llm | parser

    result = chain_insta.invoke(text)

    return result

insta_chain_runnable = RunnableLambda(insta_chain)

## Final Orchestration

In [22]:
conditional_chain = RunnableBranch(
    (lambda x: "positive" in x, chain_linkedin),
    insta_chain_runnable
)

In [23]:
final_orchestrator = prompt_template | llm_structured_output | pydantic_json_lambda | conditional_chain

In [25]:
final_orchestrator.invoke({"input": "I love Lord of the rings"})

'Here\'s a LinkedIn post for the theme "positive":\n\n**Spread Positivity in the Workplace!**\n\nAs we go about our day, it\'s easy to get caught up in stress and negativity. But I want to remind everyone that a positive attitude can make all the difference!\n\nWhen we focus on the good, we open ourselves up to new opportunities, build stronger relationships, and achieve greater success. So, let\'s make a conscious effort to spread positivity in our workplaces and communities.\n\nHere are a few ways to do just that:\n\n Show appreciation for your colleagues and their hard work\n Share a kind word or smile with someone who needs it\n Celebrate each other\'s wins, no matter how big or small\n\nRemember, positivity is contagious! Let\'s create a ripple effect of kindness and support that inspires those around us.\n\nWhat are some ways you spread positivity in your workplace? Share with me in the comments!\n\n#PositiveVibes #WorkplaceCulture #MentalHealthMatters #KindnessMatters #SuccessMi

In [26]:
final_orchestrator.invoke({"input": "I hate Lord of the rings"})

'"Sometimes you have to go through the darkness to find the light. Don\'t let negativity weigh you down, instead use it as a reminder to focus on the positive and rise above. #NegativeVibesDontLast #PositiveMindset #MentalHealthMatters"'